Pour reproduire ce notebook, installer:
* requests
* python-dotenv
* pydub -> échantillonage de l'audio
* audioop-lts -> seulement en cas d'erreur d'importation de pydub
* pyannote.audio -> diarisation; identification des locuteurs
* openai


#### Workflow: Pyannote + Découpage + LLM
* **A**: Diarisation de l'audio complet: 1 segment audio par locuteur
* **B** Découpage Audio : Associer les sorties de Pyannote en fichiers audio segmentés par locuteur:
    * Détecter les segments trop fragmentés et les consolider en 2 passes
* **C** Transcription : Envoyer chaque segment au llm pour la transcription simple (le texte uniquement).
* **D** Résolution des Noms : Utiliser LLM pour identifier les noms réels associés aux SPEAKER_XX
* **F** Construire la sortie finale: Associer chaque transcription avec les noms de locuteurs réels issus de D

#### Guide
* Créer un .env dans la racine et ajouter les entrées 'HUGGINGFACE_API_TOKEN_ASR' (à créer sur HUGGINGFACE) et 'OPENROUTER_API_KEY'
* Spécifier le lien vers votre audio ci dessous dans `AUDIO_PATH`
* Executer dans l'ordre les cellules suivantes
> Pour l'étape A.1, activez `pipeline.to(torch.device("cuda"))` si vous avez un GPU nvidia récent


In [ ]:
# A.1 Diarisation

from pyannote.audio import Pipeline
from pyannote.audio.core.io import Audio
import os
import time
import json
import torch

HUGGINGFACEHUB_API_TOKEN=os.getenv("HUGGINGFACE_API_TOKEN_ASR")
pipeline = Pipeline.from_pretrained('pyannote/speaker-diarization-community-1', token=HUGGINGFACEHUB_API_TOKEN)

AUDIO_PATH = "L-IA-notre-deuxieme-conscience.mp3"

# à activer seulement si GPU compatible
# pipeline.to(torch.device("cuda"))

# Tentative d'utiliser le chemin direct devrait fonctionner si l'audio_reader peut le lire:
try:
    t=time.time()
    output = pipeline(AUDIO_PATH)
    print(f"time process -> {time.time()-t}")
    
except ValueError as e:
    
    # Si ça échoue, vérifiez l'intégrité du fichier journal_p1.mp3 en le convertissant explicitement:
    
    import torchaudio
    # Tente de charger et de sauvegarder à nouveau le fichier pour le "réparer"
    waveform, sr = torchaudio.load(AUDIO_PATH)
    
    # Sauvegarde temporaire en WAV pour enlever les ambiguïtés du format MP3
    temp_wav_path = "/tmp/temp_pyannote.wav"
    torchaudio.save(temp_wav_path, waveform, sr)
    
    print("Fichier audio réparé en WAV. Tentative de relancer le pipeline.")
    output = pipeline(temp_wav_path)
    
    os.remove(temp_wav_path) # Nettoyage

finally:
    
    speakers_diarization=[]
    for turn, speaker in output.speaker_diarization:
        speakers_diarization.append({"speaker_id": speaker, "start": turn.start, "end": turn.end})

    print("first 5:", speakers_diarization[:5])

    # Saauvegarder
    with open("speakers_diarization.json", "w", encoding="utf-8") as f:
        json.dump(speakers_diarization, f, ensure_ascii=False, indent=2)    

/home/chougar/miniconda3/envs/asr/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[W1114 12:12:55.429742178 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1114 12:12:55.444162679 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1114 12:12:55.446335616 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1114 12:12:55.447056599 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1114 12:12:55.447791551 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1114 12:12:55.448464622 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1114 12:12:55.449186303 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1114 12:12:55.449909801 NNPACK

time process -> 6885.683816432953
SPEAKER_00 speaks between t=0.8915937500000001s and t=1.98846875s
SPEAKER_00 speaks between t=8.62034375s and t=9.46409375s
SPEAKER_00 speaks between t=10.18971875s and t=11.269718750000003s
SPEAKER_00 speaks between t=11.72534375s and t=12.535343750000003s
SPEAKER_04 speaks between t=20.11221875s and t=123.72471875000001s
SPEAKER_00 speaks between t=124.09596875000001s and t=124.97346875000001s
SPEAKER_00 speaks between t=125.66534375s and t=126.79596875000001s
SPEAKER_00 speaks between t=127.26846875000001s and t=127.94346875000001s
SPEAKER_04 speaks between t=127.94346875000001s and t=128.14596875s
SPEAKER_04 speaks between t=129.10784375s and t=224.06346875000003s
SPEAKER_02 speaks between t=144.81846875000002s and t=145.40909375s
SPEAKER_04 speaks between t=224.46846875000003s and t=248.12721875000003s
SPEAKER_02 speaks between t=248.92034375000003s and t=300.03471875s
SPEAKER_02 speaks between t=300.74346875000003s and t=359.28284375000004s
SPEAK

### Une diarisation disponible ?
En cas de diarisation disponible localement, chargez là en dé commentant la cellule suivante

In [ ]:
# A.2 Importer des traces de diarisation précédente
import json
# Load later
# with open("speakers_diarization.json", "r", encoding="utf-8") as f:
#     speakers_diarization = json.load(f)

# print("Nb segments:", len(speakers_diarization))
# i=0
# for s in speakers_diarization[:5]:
#     print(f"{s['speaker_id']}\n", f"segment {i}", ":\n", f"start; {s['start']}", f"length: {s['end']-s['start']}", "\n")
#     i+=1



Nb segments: 133
SPEAKER_00
 segment 0 :
 start; 0.8915937500000001 length: 1.0968749999999998 

SPEAKER_00
 segment 1 :
 start; 8.62034375 length: 0.84375 

SPEAKER_00
 segment 2 :
 start; 10.18971875 length: 1.0800000000000018 

SPEAKER_00
 segment 3 :
 start; 11.72534375 length: 0.8100000000000023 

SPEAKER_04
 segment 4 :
 start; 20.11221875 length: 103.61250000000001 



In [2]:
# B.1 Détection des segments mal découpés et consolidation - pass 1


from pydub import AudioSegment
import os
import io
import base64
import shutil
from pathlib import Path
import json
import hashlib


def update_segments_info(segment_id, segments_info, curr_start, curr_end, curr_speaker, filename, position):
    segments_info.append({
                "speaker_id": curr_speaker,
                "filename": filename,
                "position": position,
                "start": curr_start,
                "end": curr_end,
            })

consumed_global = set()
segments_info = []
segments_a_consolides = []

def prepare_segments_with_lookahead(speakers_diarization, max_segments=None):
    """
    Parcours _segments_info et fusionne les séries où les segments suivants
    sont courts (< lookahead_threshold_s) et du même speaker.
    Retourne segments_info (liste de métadonnées).
    """
    N = len(speakers_diarization) if max_segments is None else min(len(speakers_diarization), max_segments)
    segments_info = []
    consumed_global = set()

    i = 0
    while i < N:
        if i in consumed_global:
            i += 1
            continue

        curr = speakers_diarization[i]
        curr_start = curr["start"]
        curr_end = curr["end"]
        curr_speaker = curr["speaker_id"]

        # lookahead : construire la série à partir de i
        series = [i]
        j = i + 1
        while j < N:
            nxt = speakers_diarization[j]
            if nxt["speaker_id"] == curr_speaker:
                series.append(j)
                j += 1
            else:
                break

        # Si la série contient plusieurs éléments => fusion
        if len(series) > 1:            
            first_start = speakers_diarization[series[0]]["start"]
            last_end = speakers_diarization[series[-1]]["end"]

            for pos in series:
                consumed_global.add(pos)

            label = f"{series[0]}-{series[-1]}"
            out_path = Path(f"./segments") / f"p{label}.mp3"
            # filename=f"p{label}.mp3"
            consolidated_id = f"{curr_speaker}-{label}-{first_start}-{last_end}"
            update_segments_info(consolidated_id, segments_info, first_start, last_end, curr_speaker, out_path.name, position=i)

            print(f"Fusion (lookahead): segments {label} fusionnés")
            i = series[-1] + 1
            continue

        # Sinon série de longueur 1 -> exporter individuellement
        if (curr_end-curr_start)>=2:
            out_path = Path(f"./segments") / f"p{i}.mp3"            
            # filename=f"p{label}.mp3"
            single_id = f"{curr_speaker}-{i}-{curr_start}-{curr_end}"
            update_segments_info(single_id, segments_info, curr_start, curr_end, curr_speaker, out_path.name, position=i)

        i += 1

    return segments_info



segments_info = prepare_segments_with_lookahead(speakers_diarization, max_segments=len(speakers_diarization))
print(f"Total de {len(segments_info)} segments créés.")

Fusion (lookahead): segments 0-3 fusionnés
Fusion (lookahead): segments 5-7 fusionnés
Fusion (lookahead): segments 8-9 fusionnés
Fusion (lookahead): segments 12-21 fusionnés
Fusion (lookahead): segments 23-26 fusionnés
Fusion (lookahead): segments 30-32 fusionnés
Fusion (lookahead): segments 33-35 fusionnés
Fusion (lookahead): segments 37-58 fusionnés
Fusion (lookahead): segments 60-64 fusionnés
Fusion (lookahead): segments 66-69 fusionnés
Fusion (lookahead): segments 73-87 fusionnés
Fusion (lookahead): segments 88-90 fusionnés
Fusion (lookahead): segments 94-95 fusionnés
Fusion (lookahead): segments 99-101 fusionnés
Fusion (lookahead): segments 105-121 fusionnés
Total de 35 segments créés.


In [3]:
segments_info[:4]

[{'speaker_id': 'SPEAKER_00',
  'filename': 'p0-3.mp3',
  'position': 0,
  'start': 0.8915937500000001,
  'end': 12.535343750000003},
 {'speaker_id': 'SPEAKER_04',
  'filename': 'p4.mp3',
  'position': 4,
  'start': 20.11221875,
  'end': 123.72471875000001},
 {'speaker_id': 'SPEAKER_00',
  'filename': 'p5-7.mp3',
  'position': 5,
  'start': 124.09596875000001,
  'end': 127.94346875000001},
 {'speaker_id': 'SPEAKER_04',
  'filename': 'p8-9.mp3',
  'position': 8,
  'start': 127.94346875000001,
  'end': 224.06346875000003}]

In [ ]:
# B.2 Affiner la consolidation et découper flux audio en base64 - pass 2

def build_segments_with_lookahead(segments_info, AUDIO_PATH=AUDIO_PATH):
    """
        1. Controle et corrige les segments résiduels non fusionnés
        2. Découpe le flux audio correspondant aux segments
        3. Génère la représentation en base64
    """

    def init_folder_audio_chunks(foldername):
        # ── Configuration 
        parent_dir = Path("./")       # change to where the folder lives
        repo_path = parent_dir / foldername

        # 1 Delete the existing folder (if it exists) ───────
        if repo_path.exists():
            shutil.rmtree(repo_path)
            print(f"Deleted folder: {repo_path}")

        # 2 Re‑create the folder 
        repo_path.mkdir(parents=True, exist_ok=False)
        print(f"Created folder: {repo_path}")


    def get_base_64(segment):
        ogg_buffer = io.BytesIO()
        segment.export(ogg_buffer, format="mp3")    
        base64_segment = base64.b64encode(ogg_buffer.getvalue()).decode('utf-8')
        return base64_segment


    # output est la sortie Pyannote (la liste des segments)
    audio_file = AudioSegment.from_mp3(AUDIO_PATH)

    init_folder_audio_chunks(foldername="segments")



    print("total segments:", len(segments_info))

    segments_audio=[]
    max_turns=len(segments_info)
    for i in range(0, max_turns):
        print("traitement segment: ", i)

        curr_segment=segments_info[i]
        try:
            next_segment=segments_info[i+1]
        except:
                next_segment= None


        if next_segment!=None and curr_segment["speaker_id"]==next_segment["speaker_id"]:
            # try:
            flux = audio_file[int(curr_segment["start"]*1000): int((next_segment["end"]+0.4)*1000)]
            # except Exception as e:
            #     flux = audio_file[int(curr_segment["start"]*1000): int((next_segment["end"])*1000)]

            label_p1=curr_segment["position"]
            label_p2=next_segment["position"]
            filename=f"p{label_p1}-{label_p2}.mp3"
            out_path = Path(f"./segments") / filename
            flux.export(str(out_path), format="mp3")

            base64_segment = get_base_64(flux)

            segments_audio.append({
                "speaker_id": curr_segment["speaker_id"], 
                "filename": filename, 
                "start": curr_segment["start"],
                "end": next_segment["end"],
                "base64_audio": base64_segment,
                "positions": [curr_segment["position"], next_segment["position"]]
                }
            )
        else:
            skip=False
            if i>0:
                real_index=len(segments_audio)
                if "positions" in segments_audio[real_index-1] and curr_segment["position"] in segments_audio[real_index-1]["positions"]:
                    skip=True
                
            if skip==False and next_segment!=None:
                # try:
                flux = audio_file[int(curr_segment["start"]*1000): int((curr_segment["end"]+0.4)*1000)]
                # except Exception as e:
                #     flux = audio_file[int(curr_segment["start"]*1000): int((next_segment["end"])*1000)]

                out_path = Path(f"./segments") / curr_segment["filename"]
                flux.export(str(out_path), format="mp3")
                base64_segment = get_base_64(flux)
                segments_audio.append({
                    "speaker_id": curr_segment["speaker_id"], 
                    "filename": curr_segment["filename"], 
                    "start": curr_segment["start"],
                    "end": curr_segment["end"],
                    "base64_audio": base64_segment,
                    "positions": [curr_segment["position"]]
                    }
                )            
    return segments_audio

segments_audio=build_segments_with_lookahead(segments_info)

Deleted folder: segments
Created folder: segments
total segments: 35
traitement segment:  0
traitement segment:  1
traitement segment:  2
traitement segment:  3
traitement segment:  4
traitement segment:  5
traitement segment:  6
traitement segment:  7
traitement segment:  8
traitement segment:  9
traitement segment:  10
traitement segment:  11
traitement segment:  12
traitement segment:  13
traitement segment:  14
traitement segment:  15
traitement segment:  16
traitement segment:  17
traitement segment:  18
traitement segment:  19
traitement segment:  20
traitement segment:  21
traitement segment:  22
traitement segment:  23
traitement segment:  24
traitement segment:  25
traitement segment:  26
traitement segment:  27
traitement segment:  28
traitement segment:  29
traitement segment:  30
traitement segment:  31
traitement segment:  32
traitement segment:  33
traitement segment:  34


In [ ]:
# Print pour controler le résultat
def format_seconds_to_min(sec: float) -> str:
    minutes = int(sec // 60)
    seconds = int(sec % 60)
    milliseconds = int(round((sec - int(sec)) * 100))
    return f"{minutes}:{seconds:02d}:{milliseconds}"

for seg in segments_audio:
    print(seg["filename"], ":\n", seg["speaker_id"], "|", format_seconds_to_min(seg["start"]), "->",format_seconds_to_min(seg["end"]), "|", f"""durée: {round(seg["end"]-seg["start"],2)} s""", f"positions: {seg['positions']}", "\n-------------")

p0-3.mp3 :
 SPEAKER_00 | 0:00:89 -> 0:12:54 | durée: 11.64 s positions: [0] 
-------------
p4.mp3 :
 SPEAKER_04 | 0:20:11 -> 2:03:72 | durée: 103.61 s positions: [4] 
-------------
p5-7.mp3 :
 SPEAKER_00 | 2:04:10 -> 2:07:94 | durée: 3.85 s positions: [5] 
-------------
p8-11.mp3 :
 SPEAKER_04 | 2:07:94 -> 4:08:13 | durée: 120.18 s positions: [8, 11] 
-------------
p12-21.mp3 :
 SPEAKER_02 | 4:08:92 -> 6:39:18 | durée: 150.25 s positions: [12] 
-------------
p22.mp3 :
 SPEAKER_04 | 6:39:50 -> 6:48:93 | durée: 9.43 s positions: [22] 
-------------
p23-26.mp3 :
 SPEAKER_01 | 6:49:40 -> 9:11:67 | durée: 142.27 s positions: [23] 
-------------
p27.mp3 :
 SPEAKER_04 | 9:12:43 -> 9:36:38 | durée: 23.95 s positions: [27] 
-------------
p28.mp3 :
 SPEAKER_03 | 9:36:95 -> 11:13:98 | durée: 97.03 s positions: [28] 
-------------
p29.mp3 :
 SPEAKER_04 | 11:14:10 -> 11:58:42 | durée: 44.31 s positions: [29] 
-------------
p30-32.mp3 :
 SPEAKER_01 | 11:58:52 -> 13:55:4 | durée: 116.52 s positions: 

#### Bugs constatés
Chevauchements:
* p92-94.mp3 et p94-97.mp3 
* p94-97.mp3 et p97-99.mp3

C bis 3.
1. pass 1 LLM pour transcript brut segments audio pyannote -> text, sans se soucier des locuteurs
2. pass 2 LLM pour identification des locuteurs, avec comme input:
    * transcripts audio issus de step 1 (200 premiers caract) dans un contexte global + speakers_id issues de pyannote

In [52]:
# C; appel LLM pour transcript brut segments audio pyannote

import requests
import base64
import os
import json
from dotenv import load_dotenv

load_dotenv(".env")

def llm_transcribe_segment(base64_audio_data: str, id_segment=int) -> str:
    """
    Transcrit un segment audio
    """

    API_KEY_REF=os.getenv("OPENROUTER_API_KEY")    
    
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY_REF}",
        "Content-Type": "application/json"
    }

    
    # --- 2. Construction du Prompt ---
    # v1
    # llm_prompt = f"""
    #     Ceci est un segment audio d'une émission de radio, diarisé avec pyannote.
    #     Votre tâche est de fournir la transcription exacte du segment.
    # """

    # v2
    llm_prompt="""
        Votre tâche est de fournir une transcription méticuleuse et intégrale de ce segment audio issu d'une émission de radio.

        Instructions importantes :
        1.  **Précision Maximale :** Transcrivez le discours mot pour mot, y compris les hésitations (comme "euh", "hmm").
        2.  **Attention aux Noms Propres :** Portez une attention toute particulière à la retranscription correcte et complète des noms de personnes, de lieux, ou de marques. Ne les tronquez pas. Si un nom est prononcé, faites votre possible pour le retranscrire en entier, même si la prononciation est rapide.
        3.  **Ponctuation :** Utilisez une ponctuation (virgules, points, points d'interrogation) qui reflète fidèlement le flux et l'intonation de la parole pour rendre le texte lisible.
        4.  **Termes Techniques :** Faites attention aux termes techniques ou au jargon potentiellement liés au sujet de l'émission.

        Transcrivez le segment audio maintenant.
    """
    
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": llm_prompt},
                {"type": "input_audio", "input_audio": {"data": base64_audio_data, "format": "mp3"}}
            ]
        }
    ]

    # --- 3. Appel à l'API et Extraction (utiliser l'extraction robuste) ---
    
    try:
        response = requests.post(url, headers=headers, json={"model": "google/gemini-2.5-flash", "messages": messages})
        response.raise_for_status()
        raw_output = response.json()['choices'][0]['message']['content']
        return raw_output        

    except Exception as e:
        print(f"Erreur de traitement LLM pour segment {id_segment}: {e}")
        # Retourner un objet par défaut pour continuer le flux
        return {"Transcription": f"[Erreur: {e}]", "segment": id_segment}


# --- 2. Mise à Jour de la Boucle de Traitement ---

# Assumons que segments_info contient les segments de l'Étape A
# La clé 'data_llm' sera ajoutée au fur et à mesure.
# limit =5
limit=len(segments_audio)

for i in range(0, limit):    

    # Envoi à la fonction de transcription et d'identification
    segments_audio[i]["transcription"] = llm_transcribe_segment(
        base64_audio_data=segments_audio[i]["base64_audio"], 
        id_segment=i,        
    )


In [53]:
# Print de controle
def format_seconds_to_min(sec: float) -> str:
    minutes = int(sec // 60)
    seconds = int(sec % 60)
    milliseconds = int(round((sec - int(sec)) * 100))
    return f"{minutes}:{seconds:02d}:{milliseconds}"

i=1
for seg in segments_audio[:10]:
    # print(s["speaker_id"], s["transcription"], "\n========")
    print(seg["speaker_id"], f"""\n{seg["filename"]} - {format_seconds_to_min(seg["start"])}---> {format_seconds_to_min(seg["end"])}\n""", 
        f"""Transcription:\n {seg["transcription"]}""", "\n")
    i+=1

SPEAKER_00 
p0-3.mp3 - 0:00:89---> 0:12:54
 Transcription:
 France Culture France Culture sans préjugés, Nathan Devert. 

SPEAKER_04 
p4.mp3 - 0:20:11---> 2:03:72
 Transcription:
 Comment expliquer les progrès phénoménaux que semble avoir accomplis l'intelligence artificielle au cours de ces dernières années. Depuis l'apparition de chat GPT en novembre 2022, rapidement suivi par d'autres agents conversationnels. Cette révolution technologique aux multiples aspects paraît désormais capable d'exécuter de nombreuses tâches intellectuelles sur lesquelles l'esprit humain pensait jusqu'à alors exercer un monopole. Écrire des articles, synthétiser des documents, traiter des données dans n'importe quel domaine, diagnostiquer une maladie, rédiger une dissertation, ou pourquoi pas, un scénario de film. Non contente de révolutionner le monde du travail, ses prouesses stupéfiantes de la technique soulèvent une interrogation majeure dans le domaine de la philosophie de l'esprit. Faut-il en déduire 

In [54]:
# D. identifier les noms des locteurs - 

import requests
import json
import base64
import os
import json
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(".env")

def llm_identify_speakers(context: list) -> str:
    """
    Identifie les locuteurs
    """

    API_KEY_REF=os.getenv("OPENROUTER_API_KEY")    
    
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY_REF}",
        "Content-Type": "application/json",
    }

    
    # --- 2. Construction du Prompt ---
    prompt=f"""
        Ta tâche est d'identifier le nom **le plus complet et plausible** ET le rôle (métier, activité, fonction) de chaque locuteur, en synthétisant les informations de tous les extraits. Tu dois aussi qualifier la nature du son s'il ne s'agit pas d'une personne.

        Note que:
        *   Ces extraits se présentent sous la forme "extrait - speaker_id", où speaker_id est issu d'une diarisation pyannote.
        *   Ces extraits suivent un ordre chronologique strict.
        *   Ne sont fournis que les 300 premiers et derniers caractères de chaque extrait.
        *   Un locuteur peut être une personne, un témoin, un jingle, une annonce, etc.

        **Règles d'identification importantes:**
        1.  **Annonces et Jingles :** Si un extrait est un jingle ou une voix-off, identifie le `nom` comme "Annonce" ou "Jingle" et le `rôle` comme "Identifiant sonore". Le locuteur est celui qui parle, pas celui dont on parle.
        2.  **L'animateur :** La personne qui introduit les invités, pose les questions et distribue la parole est l'animateur. Son `rôle` est "Animateur" ou "Présentateur".
        3.  **Les invités :** Le nom et le rôle d'un invité sont souvent révélés lorsque l'animateur le présente ou s'adresse à lui (ex: "...avec le **mathématicien et philosophe Daniel Angler**...").
        4.  **Rôle non spécifié :** Si un locuteur est une personne mais que son métier n'est pas mentionné, utilise "Invité" ou "Intervenant" comme `rôle`.
        **5. Consolidation et Correction du Nom : Un nom peut être tronqué, mal orthographié ou varier d'un extrait à l'autre à cause d'erreurs de transcription. Tu dois analyser **toutes** les mentions relatives à un même locuteur (`speaker_id`) à travers **tous** les extraits pour reconstituer le nom complet le plus plausible.**
            *   **Exemple de logique à suivre :** Si pour un même locuteur tu trouves "Jean-Pierre Dupo" (tronqué), puis "Jean-Pierre Dupont" (complet), et plus tard "Jean-Pierre Dumont" (faute de frappe probable), tu dois synthétiser ces informations pour conclure que le nom correct et complet est **"Jean-Pierre Dupont"**.
        
        Contexte:
        {context}
        

        Identifie puis retourne une association `speaker_id` -> objet JSON contenant les clés `name` et `role`.

        Respecte une sortie JSON valide.
        Exemple:
        ```json
            {{
                "SPEAKER_00": {{
                    "name": "Annonce",
                    "role": "Jingle d'émission"
                }},
                "SPEAKER_01": {{
                    "name": "Lawrence d'Arabie",
                    "role": "Soldat, explorateur, aventurier"
                }},
                "SPEAKER_02": {{
                    "name": "Daniel Biguenet",
                    "role": "Chauffeur routier"
                }}
            }}
    """
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
            ]
        }
    ]

    # --- 3. Appel à l'API et Extraction (utiliser l'extraction robuste) ---
    
    try:
        # response = requests.post(url, headers=headers, json={"model": "google/gemini-2.5-flash", "messages": messages})

        client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=API_KEY_REF,
        )

        response = client.chat.completions.create(
            extra_headers={
                "HTTP-Referer": "Podcast_augmente", # Optional. Site URL for rankings on openrouter.ai.
                "X-Title": "Podcast_augmente", # Optional. Site title for rankings on openrouter.ai.
            },
            extra_body={
                "Projet": "Podcast_augmente",
                "reasoning": {"enabled": False}
            },
            # model="meta-llama/llama-4-maverick",
            # model="deepseek/deepseek-v3.2-exp",
            model='deepseek/deepseek-v3.1-terminus',
            messages=[
              {
                "role": "user",
                "content": [
                  {
                    "type": "text",
                    "text": prompt
                  }
                ]
              }
            ],
            temperature=0
        )
        
        # response.raise_for_status()        
        # raw_output = response.json()['choices'][0]['message']['content']

        raw_output=response.choices[0].message.content
        
        # Extraction du JSON
        json_resp = raw_output[raw_output.find("```json") + 7: raw_output.rfind("}") + 1].strip()

        return json.loads(json_resp) 

    except Exception as e:
        print(f"Erreur de traitement : {e}")
        # Retourner un objet par défaut pour continuer le flux
        # return {"Transcription": f"[Erreur: {e}]", "segment": id_segment}


# --- 2. Mise à Jour de la Boucle de Traitement ---

# Assumons que segments_info contient les segments de l'Étape A
# La clé 'data_llm' sera ajoutée au fur et à mesure.


context=""
for i in range(0, limit):    
    if len(segments_audio[i]["transcription"])>300:
        transcript=f'{segments_audio[i]["transcription"][:300]}...{segments_audio[i]["transcription"][-300:]}'
    else:
        transcript=segments_audio[i]["transcription"]

    context+=f"""
        transcription:
            {transcript}

        "SPEAKER_ID": {segments_audio[i]["speaker_id"]}\n
    """

# Envoi à la fonction de transcription et d'identification
raw_locuteurs_identities = llm_identify_speakers(
    context=context
)

print("Contexte:\n", context)
raw_locuteurs_identities

Contexte:
 
        transcription:
            France Culture France Culture sans préjugés, Nathan Devert.

        "SPEAKER_ID": SPEAKER_00

    
        transcription:
            Comment expliquer les progrès phénoménaux que semble avoir accomplis l'intelligence artificielle au cours de ces dernières années. Depuis l'apparition de chat GPT en novembre 2022, rapidement suivi par d'autres agents conversationnels. Cette révolution technologique aux multiples aspects paraît déso...ligence artificielle, notre deuxième conscience, à l'occasion de la fête de la science, j'aurai le plaisir d'en débattre sans préjugés au côté du mathématicien et philosophe Daniel Angler, de la professeur en intelligence artificielle et chercheuse Laurence de Ville et du philosophe Valentin Husson.

        "SPEAKER_ID": SPEAKER_04

    
        transcription:
            France Culture, sans préjuger. Nathan Devers.

        "SPEAKER_ID": SPEAKER_00

    
        transcription:
            Bonjour Laurence d

{'SPEAKER_00': {'name': 'Annonce', 'role': "Jingle d'émission"},
 'SPEAKER_01': {'name': 'Laurence Devillers',
  'role': 'Professeure en intelligence artificielle, chercheuse au CNRS, présidente de la Fondation Blaise Pascal'},
 'SPEAKER_02': {'name': 'Daniel Andler',
  'role': 'Mathématicien et philosophe'},
 'SPEAKER_03': {'name': 'Valentin Husson', 'role': 'Philosophe'},
 'SPEAKER_04': {'name': 'Nathan Devers', 'role': 'Animateur'}}

In [43]:
def correct_speakers_names(raw_locuteurs_identities: dict, fail_try_max: int=6) -> dict:
    """
        Inputs:
            * raw_locuteurs_identities(dict): dictionnaire des locuteurs identifiés lors de la transcription audio du LLM
            * fail_try_max(int): nombre de requêtes successives en cas d'échec de l'API
        Output:
            * locuteurs_identities(dict): dictionnaire des locuteurs dont le nom est corrigé
    """
    from ddgs import DDGS
    import time
   
    def call_llm_validation(web_results: list, name_locuteur: str) -> dict:
        import requests
        import json
        import base64
        import os
        import json
        from dotenv import load_dotenv

        load_dotenv(".env")


        API_KEY_REF=os.getenv("OPENROUTER_API_KEY")    
        
        url = "https://openrouter.ai/api/v1/chat/completions"
        headers = {
            "Authorization": f"Bearer {API_KEY_REF}",
            "Content-Type": "application/json"
        }

        formated_web_results=[f'{el["title"]}. {el["body"][:100]}' for el in web_results]

        print(f"\nFormated_web_results pour '{name_locuteur}':\n")        
        for el, i in zip(web_results, list(range(0,len(web_results)))):
            print(f"Resultat {i}:\n {el}\n-----")

        prompt=f"""
            Ta tâche est de corriger une orthographe potentiellement erronée d'un nom de personne en te basant sur des résultats de recherche web.

            Tu recevras trois informations :
            1.  'nom_provisoire': Le nom tel qu'il a été identifié, potentiellement mal orthographié.
            2.  'role_associe': Le métier, la fonction ou l'activité de la personne, qui sert de critère de validation.
            3.  `contexte_web`: La concaténation du titre et du corps des résultats d'une recherche web effectuée avec le nom et le rôle.

            **Règles de décision :**
            1.  **ANALYSE :** Cherche des noms complets de personnes dans le `contexte_web`.
            2.  **VALIDATION :** Pour chaque nom trouvé, vérifie si le texte associé confirme **clairement** que cette personne a bien le `role_associe`. La correspondance du rôle est plus importante que la similarité du nom.
            3.  **CAS DE SUCCÈS :** Si tu trouves un nom dans le `contexte_web` dont la description (métier, activité) correspond au `role_associe`, alors ce nom est le nom corrigé.
            4.  **CAS D'ÉCHEC :** Si l'une des conditions suivantes est remplie, alors aucune correction fiable n'est possible :
                *   Le `contexte_web` est vide ou ne contient aucune information pertinente.
                *   Le `contexte_web` mentionne des personnes, mais **aucune** ne correspond au `role_associe` (cas d'homonymes).
                *   La correspondance entre le rôle et la description est trop faible ou ambiguë.

            **Format de sortie :**
            Tu dois retourner **uniquement** un objet JSON valide avec une seule clé : `nom_corrige`.
            *   Si la correction est réussie (cas de succès), la valeur sera le nom correct trouvé dans `contexte_web`.
            *   Si la correction n'est pas possible (cas d'échec), la valeur doit être le `nom_provisoire` original.
            *   Tu dois renvoyer un score de confiance de ta correction allant de 0 à 10

            ---
            **EXEMPLES**
            ---

            **Exemple 1 : Correction réussie**

            `nom_provisoire`: "Daniel Carvalo"
            `role_associe`: "Footballer"
            `contexte_web`: 
                Daniel Carvalho — Wikipédia
                Daniel Carvalho est un footballeur brésilien né le 10 mars 1981 à São Paulo. Il évolue au poste de défenseur....

                Daniel Carvalho - FootballDatabase
                Daniel Carvalho - Stats et palmarès - Toutes les statistiques de Daniel Carvalho évoluant au poste de défenseur central

            **Sortie attendue :**
            ```json
            {{
                "nom_corrige": "Daniel Carvalho",
                "confiance_score": 10
            }}
            ```

            ---

            **Exemple 2 : Cas d'homonyme (échec de la validation)**

            `nom_provisoire`: "Paul Bernard"
            `role_associe`: "Climatologue"
            `contexte_web`:
                Paul Bernard, votre boulanger-pâtissier à Lyon
                Artisan boulanger passionné, je vous propose des pains au levain et des viennoiseries...

                Maître Paul Bernard - Avocat au barreau de Paris
                Le cabinet de Maître Bernard est spécialisé en droit des sociétés et droit fiscal...

            **Sortie attendue :**
            ```json
            {{
                "nom_corrige": "Paul Bernard",
                "confiance_score": 5
            }}
            ```

            ---

            **Exemple 3 : Pas de personnalité publique (échec)**
                `nom_provisoire`: "Sophie Martin"
                `role_associe`: "Témoin"
                `contexte_web`: ""

            **Sortie attendue :**
            ```json
            {{
                "nom_corrige": "Sophie Martin",
                "confiance_score": 5
            }}
            ```

            ---
            **À TRAITER MAINTENANT**
            ---

            `nom_provisoire`: {locuteur["name"]}
            `role_associe`: {locuteur['role']}
            `contexte_web`: {formated_web_results}
        """

        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                ]
            }
        ]

        # --- 3. Appel à l'API et Extraction (utiliser l'extraction robuste) ---
        
        try:
            response = requests.post(url, headers=headers, json={"model": "google/gemini-2.5-flash", "messages": messages})
            response.raise_for_status()
            raw_output = response.json()['choices'][0]['message']['content']
            
            # Extraction du JSON
            json_resp = raw_output[raw_output.find("```json") + 7: raw_output.rfind("}") + 1].strip()

            return json.loads(json_resp) 

        except Exception as e:
            print(f"Erreur de traitement : {e}")


    locuteurs_identities={}
    for speaker_id, locuteur in raw_locuteurs_identities.items():
        if locuteur['role'] in ["Identifiant sonore", "Annonce sonore", "Jingle d'émission", "Témoin", "Passant", "Manifestant", "Anonyme"]:
            locuteurs_identities[speaker_id]= {'nom_corrige': locuteur["name"], 'confiance_score': 10}
            continue
        
        web_results=[]
        acquired_results=False
        tentative=1
        while acquired_results==False and tentative<6:
            try:
                _web_results = DDGS().text(
                    f"{locuteur['name']}, {locuteur['role']}", 
                    max_results=10, 
                    region="fr-fr", 
                    language="fr", 
                    backend='duckduckgo'
                )
                web_results+=_web_results
                time.sleep(1)
                acquired_results=True
            except Exception as e:
                print(f"Erreur recherche web sources pour {locuteur['name']}:\n{e}\nTentative N° {tentative}")
                acquired_results=False
                tentative+=1
                time.sleep(1)

        locuteurs_identities[speaker_id]=call_llm_validation(web_results, locuteur["name"])

    return locuteurs_identities
        


# test_raw_locuteurs_identities= {'SPEAKER_00': {'name': 'Daniel Hangler', 'role': 'Mathématicien et philosophe'}}
# test_raw_locuteurs_identities= {'SPEAKER_00': {'name': 'Valentin Husser', 'role': 'Philosophe'}}
test_raw_locuteurs_identities= {'SPEAKER_00': {'name': 'Annonce', 'role': 'Identifiant sonore'}}
locuteurs_identities= correct_speakers_names(test_raw_locuteurs_identities)
locuteurs_identities

{'SPEAKER_00': {'nom_corrige': 'Annonce', 'confiance_score': 10}}

In [55]:
locuteurs_identities= correct_speakers_names(raw_locuteurs_identities)
locuteurs_identities



Formated_web_results pour 'Laurence Devillers':

Resultat 0:
 {'title': 'Laurence Devillers — Wikipédia', 'href': 'https://fr.wikipedia.org/wiki/Laurence_Devillers', 'body': "Laurence Devillers, née le 8 octobre 1962 à Châtillon-sur-Seine, est enseignante- chercheuse en informatique appliquée aux sciences sociales, en poste à l' Université Paris-Sorbonne depuis 2011."}
-----
Resultat 1:
 {'title': 'Laurence Devillers - Sorbonne Université', 'href': 'https://www.sorbonne-universite.fr/portraits/laurence-devillers', 'body': "Laurence Devillers, professeure au LISN 2, fait partie de la génération de chercheurs qui a accompagné les débuts de l'IA. Avec en tête que cette technologie peut nous en apprendre beaucoup sur la naissance du langage."}
-----
Resultat 2:
 {'title': "Laurence Devillers : « L'intelligence artificielle n'est ni magique, ni ...", 'href': 'https://www.apprentis-auteuil.org/actualites/societe/laurence-devillers-lintelligence-artificielle-nest-ni-magique-ni-consciente-ni-

{'SPEAKER_00': {'nom_corrige': 'Annonce', 'confiance_score': 10},
 'SPEAKER_01': {'nom_corrige': 'Laurence Devillers', 'confiance_score': 10},
 'SPEAKER_02': {'nom_corrige': 'Daniel Andler', 'confiance_score': 10},
 'SPEAKER_03': {'nom_corrige': 'Valentin Husson', 'confiance_score': 10},
 'SPEAKER_04': {'nom_corrige': 'Nathan Devers', 'confiance_score': 10}}

In [56]:
for s in segments_audio:
    if s["speaker_id"] in locuteurs_identities.keys():
        s["locuteur"]=locuteurs_identities[s["speaker_id"]]["nom_corrige"]
    else:
        s["locuteur"]="Non identifié"


In [63]:
def format_seconds_to_hhmmss(sec: float) -> str:
    total_seconds = int(sec)  # ignore milliseconds
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"


full_text=""
i=1
for seg in segments_audio:    
    full_text+=f"""
        {format_seconds_to_hhmmss(seg["start"])} ---> {format_seconds_to_hhmmss(seg["end"])}
        {seg["locuteur"]}: 
        {seg["transcription"]}
    """
    i+=1
# Saauvegarder

with open("output_audio_text.txt", "w", encoding="utf-8") as f:
    f.write(full_text)

print(full_text)



        00:00:00 ---> 00:00:12
        Annonce: 
        France Culture France Culture sans préjugés, Nathan Devert.
    
        00:00:20 ---> 00:02:03
        Nathan Devers: 
        Comment expliquer les progrès phénoménaux que semble avoir accomplis l'intelligence artificielle au cours de ces dernières années. Depuis l'apparition de chat GPT en novembre 2022, rapidement suivi par d'autres agents conversationnels. Cette révolution technologique aux multiples aspects paraît désormais capable d'exécuter de nombreuses tâches intellectuelles sur lesquelles l'esprit humain pensait jusqu'à alors exercer un monopole. Écrire des articles, synthétiser des documents, traiter des données dans n'importe quel domaine, diagnostiquer une maladie, rédiger une dissertation, ou pourquoi pas, un scénario de film. Non contente de révolutionner le monde du travail, ses prouesses stupéfiantes de la technique soulèvent une interrogation majeure dans le domaine de la philosophie de l'esprit. Faut-il en dé

#### Reste à faire
* P1: correction des noms de locuteurs dans le corps de la transcription
* P2: Transcription avec whisper
